In [10]:
import zipfile, os
import pandas as pd

In [11]:
_file = os.path.join('Data', 'Data.zip')
zf = zipfile.ZipFile(_file)

In [12]:
filenames = zf.namelist()

In [19]:
pd.read_csv(zf.open(filenames[0]))

,Jersey,Player,Yr,Pos,GP,GS,G,RushNetYards,ReceivingYards,IntRYds,PuntRetYds,FRetYds,KORetYds,APY,All-PurposeYardsG,stat,year,team
0,17,"Rhodes, J'Marcus",Jr,DB,12,3,12,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
1,0K,"Crump, Koron",Jr,LB,12,8,12,NaN,NaN,59.0,NaN,42.0,NaN,101,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
2,0W,"Wicker, Jojo",So,DL,12,11,12,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
3,12,"White, Tim",Sr,WR,12,7,12,-4.0,713.0,NaN,254.0,NaN,709.0,1672,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
4,92,"Topps, Tramel",Sr,DL,12,0,12,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
5,22,"Ralston, Nick",Fr,RB,12,0,12,139.0,10.0,NaN,NaN,NaN,NaN,149,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
6,8,"Hayes, De'Chavon",Sr,DB,12,9,12,NaN,NaN,56.0,NaN,NaN,30.0,86,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
7,90,"Smallwood, Tashon",Jr,DL,12,12,12,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
8,95,"Wren, Renell",So,DL,12,1,12,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
9,7,"Ballage, Kalen",Jr,RB,12,6,12,536.0,469.0,NaN,-6.0,NaN,409.0,1408,0.0,All-Purpose Yards,2016-17,Arizona St. Sun Devils
